In [1]:
import numpy as np                                     
import matplotlib.pyplot as plt                      
import seaborn as sns                                
from iminuit import Minuit                      
import sys             
from sympy import *                           
from scipy import stats
from scipy.stats import binom
from scipy.special import binom as binomcoef
from scipy.special import expit
import pandas as pd
import os
from importlib import reload

plt.style.use("../KD_style.mplstyle")

In [6]:
def acceptreject(func, xmin, xmax, ymin, ymax, N_points, **kwargs):
    # Recursive function to do accept/reject monte carlo simulation
    xran = np.random.uniform(xmin, xmax, N_points)
    yran = np.random.uniform(ymin, ymax, N_points)
    yfunc = func(xran, **kwargs)
    xkeep = xran[yran < yfunc]
    ykeep = yran[yran < yfunc]
    missing = N_points - len(xkeep)
    if missing > 0:
        xrest, yrest, all_xrest, all_yrest, tries = acceptreject(func, xmin, xmax, ymin, ymax, missing, **kwargs)
    else:
        xrest = np.array([])
        yrest = np.array([])
        all_xrest = np.array([])
        all_yrest = np.array([])
        tries = 0
    finalx = np.append(xkeep, xrest)
    finaly = np.append(ykeep, yrest)
    allx = np.append(xran, all_xrest)
    ally = np.append(yran, all_yrest)
    finaltries = N_points + tries
    return finalx, finaly, allx, ally, finaltries

def acceptrejectdata(func, xmin, xmax, ymin, ymax, N_points, **kwargs):
    x, y, all_x, all_y, tries = acceptreject(func, xmin, xmax, ymin, ymax, N_points, **kwargs)
    eff = N_points/tries
    area = (xmax - xmin) * (ymax - ymin) * eff
    return x, y, area, all_x, all_y, eff, tries

def num_int2(func, xmin, xmax, points = 1000000, **kwargs):
    # This method is better than num_int1
    x = np.linspace(xmin, xmax, points)
    y = func(x, **kwargs)
    y_ave = y[1:] + (y[1:] - y[:-1])/2
    width = (xmax-xmin)/(points-1)
    area = np.sum(y_ave)*width
    return area

In [ ]:
data = pd.read_csv("./ParameterEstimation_Ex1.txt", names = ["x"])

In [8]:
xmin = -0,95
xmax = 0.95

def func(x, alpha, beta):
    return (1+alpha*x+beta*x**2)/((beta*(-xmin**3+xmax**3)/3)+(alpha*(-xmin**2+xmax**2)/2)+(xmax-xmin))
    
num_int2(func, xmin, xmax, alpha = 0.5, beta = 0.5)

TypeError: unsupported operand type(s) for ** or pow(): 'tuple' and 'int'